In [12]:
# Enhanced residue-level feature extraction from mmCIF using Bio.PDB
import os
import numpy as np
import pandas as pd
from Bio.PDB import MMCIFParser, NeighborSearch
from Bio.PDB.Polypeptide import is_aa
from pathlib import Path
from collections import Counter
from scipy.spatial.distance import pdist

In [13]:
import os
from pathlib import Path
from Bio.PDB import MMCIFParser, NeighborSearch
from Bio.PDB.Polypeptide import is_aa
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist
import gzip

def extract_biopdb_features(
    cif_dir, 
    output_csv='../data/raw/sample_non_biolip_residue_structural_features_biopdb.csv'
):
    parser = MMCIFParser(QUIET=True)
    all_features = []
    cutoff_radii = [4, 6, 8, 10]

    cif_files = sorted([f for f in os.listdir(cif_dir) if f.endswith('.cif.gz')])
    print(f"Processing {len(cif_files)} CIF files")

    for file in cif_files:
        pdb_id = Path(file).stem.split('.')[0].lower()  # 'pdbXXXX.ent.cif.gz' -> 'pdbXXXX.ent' -> 'XXXX'
        path = os.path.join(cif_dir, file)

        try:
            with gzip.open(path, mode='rt') as handle:
                structure = parser.get_structure(pdb_id, handle)

            model = structure[0]
            atoms = list(model.get_atoms())
            ns = NeighborSearch(atoms)

            for chain in model:
                residues = [r for r in chain if is_aa(r, standard=True)]
                for i, residue in enumerate(residues):
                    hetfield, resseq, icode = residue.get_id()
                    atoms_list = list(residue.get_atoms())
                    coords = np.array([atom.coord for atom in atoms_list])
                    if coords.size == 0:
                        continue

                    b_factors = [atom.get_bfactor() for atom in atoms_list]
                    occupancy = [atom.get_occupancy() for atom in atoms_list]
                    mean_b = np.mean(b_factors)
                    std_b = np.std(b_factors)
                    mean_occ = np.mean(occupancy)
                    num_atoms = len(atoms_list)
                    num_heavy_atoms = sum(1 for atom in atoms_list if atom.element != 'H')
                    num_sidechain_atoms = sum(1 for atom in atoms_list if atom.get_id() not in ['N', 'CA', 'C', 'O'])

                    centroid = np.mean(coords, axis=0)
                    if len(coords) > 1:
                        inter_dists = pdist(coords)
                        mean_dist = np.mean(inter_dists)
                        std_dist = np.std(inter_dists)
                        radius_of_gyration = np.sqrt(np.mean(np.sum((coords - centroid) ** 2, axis=1)))
                    else:
                        mean_dist = std_dist = radius_of_gyration = 0.0

                    bounding_box_volume = np.prod(np.ptp(coords, axis=0))
                    residue_radius = np.max(np.linalg.norm(coords - centroid, axis=1))

                    contact_counts = {}
                    for r in cutoff_radii:
                        neighbors = ns.search(centroid, r, level='R')
                        contact_counts[f'contact_number_{r}A'] = sum(1 for nbr in neighbors if is_aa(nbr, standard=True) and nbr != residue)

                    neighbor_atoms = ns.search(centroid, 10.0, level='A')
                    neighbor_heavy_atoms = [atom for atom in neighbor_atoms if atom.element != 'H' and atom not in atoms_list]
                    closest_neighbor_dist = min([np.linalg.norm(centroid - atom.coord) for atom in neighbor_heavy_atoms], default=np.nan)
                    avg_neighbor_distance = np.mean([np.linalg.norm(centroid - atom.coord) for atom in neighbor_heavy_atoms]) if neighbor_heavy_atoms else np.nan

                    prev_res = residues[i - 1].get_resname() if i > 0 else 'NA'
                    curr_res = residue.get_resname()
                    next_res = residues[i + 1].get_resname() if i < len(residues) - 1 else 'NA'
                    position_in_chain = i
                    is_small = int(curr_res in {'GLY', 'ALA', 'SER'})

                    row = {
                        'pdb_id': pdb_id,
                        'chain_id': chain.id,
                        'residue_number': resseq,
                        'insertion_code': icode,
                        'residue_name': curr_res,
                        'centroid_x': centroid[0],
                        'centroid_y': centroid[1],
                        'centroid_z': centroid[2],
                        'mean_bfactor': mean_b,
                        'std_bfactor': std_b,
                        'mean_occupancy': mean_occ,
                        'num_atoms': num_atoms,
                        'num_heavy_atoms': num_heavy_atoms,
                        'num_sidechain_atoms': num_sidechain_atoms,
                        'mean_intra_atom_dist': mean_dist,
                        'std_intra_atom_dist': std_dist,
                        'radius_of_gyration': radius_of_gyration,
                        'residue_radius': residue_radius,
                        'bounding_box_volume': bounding_box_volume,
                        'closest_neighbor_dist': closest_neighbor_dist,
                        'avg_neighbor_distance': avg_neighbor_distance,
                        'prev_res': prev_res,
                        'next_res': next_res,
                        'position_in_chain': position_in_chain,
                        'is_small': is_small,
                        **contact_counts
                    }
                    all_features.append(row)

        except Exception as e:
            print(f"[ERROR] {pdb_id}: {e}")

        print(f"\nProcessing {pdb_id} - total residues so far: {len(all_features)}", end='\r')

    df = pd.DataFrame(all_features)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Features saved to {output_csv}")
    return df


In [14]:
if __name__ == "__main__":
    cif_folder = "../data/raw/pdb_cifs"
    extract_biopdb_features(cif_folder)

Processing 5000 CIF files

Processing pdb131l - total residues so far: 162
Processing pdb161l - total residues so far: 324
Processing pdb194l - total residues so far: 453
Processing pdb1a09 - total residues so far: 663
Processing pdb1a2t - total residues so far: 797
Processing pdb1a2w - total residues so far: 1045
Processing pdb1a4w - total residues so far: 1327
Processing pdb1a6j - total residues so far: 1634
Processing pdb1a7n - total residues so far: 1857
Processing pdb1a8k - total residues so far: 2253
Processing pdb1ab2 - total residues so far: 2362
Processing pdb1abs - total residues so far: 2516
Processing pdb1abz - total residues so far: 2554
Processing pdb1aci - total residues so far: 2630
Processing pdb1ad2 - total residues so far: 2854
Processing pdb1ae8 - total residues so far: 3152
Processing pdb1aeb - total residues so far: 3443
Processing pdb1aeq - total residues so far: 3734
Processing pdb1afu - total residues so far: 3982
Processing pdb1agt - total residues so far: 402